In [0]:
#!pip install syft

In [0]:
import numpy as np
import torch

In [3]:
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8)

In [4]:
db.shape

torch.Size([5000])

In [0]:
remove_index = 2

In [6]:
db[0:2]

tensor([0, 1], dtype=torch.uint8)

In [7]:
db[3:]

tensor([1, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8)

In [0]:
def get_parallel_db(db, remove_index):
  
  return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [9]:
get_parallel_db(db, 0)[0:5]

tensor([1, 1, 1, 0, 0], dtype=torch.uint8)

In [10]:
get_parallel_db(db, 0).shape

torch.Size([4999])

In [0]:
def get_parallel_dbs(db):
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
  
  return parallel_dbs

In [0]:
pdbs = get_parallel_dbs(db)

In [13]:
pdbs

[tensor([1, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8),
 tensor([0, 1, 1,  ..., 1, 1

In [0]:
def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(20)

In [16]:
db

tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       dtype=torch.uint8)

In [17]:
pdbs

[tensor([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
        dtype=torch.uint8),
 tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,

**Project 3 - Evaluating Differential Privacy of a Function**

Intuitively, we want to check whether the query of the database is releasing any "private" information. So, here we evaluate whether the output of the query changes when we remove someone from the database. Specifically, we want to evaluate the maximum amount the query changes when someone is removed. To find how much information is leaked, we iterate over each person in the database and measure the difference in the output relative to when we query the entire database.

In [0]:
db, pdbs = create_db_and_parallels(5000)

In [0]:
#db

In [0]:
def query(db):
  return db.sum()

In [0]:
full_query_result = query(db)

In [0]:
max_distance = 0
for pdb in pdbs:
  pdb_result = query(pdb)
  
  db_distance = torch.abs(pdb_result - full_query_result)
  
  if(db_distance > max_distance):
    max_distance = db_distance

In [23]:
max_distance

tensor(1)

In [0]:
def sensitivity(query, n_entries=1000):
  db, pdbs = create_db_and_parallels(n_entries)
  
  full_db_result = query(db)
  
  max_dist = 0
  for i in pdbs:
    pdb_result = query(i)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > max_dist):
      max_dist = db_distance
      
  return max_dist  
    

In [0]:
def query(db):
  return db.float().mean()

In [26]:
sensitivity(query)

tensor(0.0005)

In [27]:
0.5 / 1000

0.0005

**Calculate L1 Sensitivity for Threshold**

> Compute sum over the database and check if it greater than a certain threshold



> Create 10 databases of size 10 and threshold 5 and calculate the sensitivity of the function.



> Reinitialize the database 10 times and calculate sensitivity each time








In [0]:
def query(db, threshold=5):
  return (db.sum() > threshold).float()

In [29]:
db, pdbs = create_db_and_parallels(10)
db.sum()                          

tensor(5)

In [30]:
db.sum() > 5

tensor(0, dtype=torch.uint8)

In [31]:
query(db)

tensor(0.)

In [32]:
for i in range(10):

  sens_f = sensitivity(query, n_entries=10)
  print(sens_f)

0
0
0
0
0
0
0
0
0
tensor(1.)


**Perform a Differencing Attack on Row 10**

In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index=10)

In [35]:
db[10]

tensor(0, dtype=torch.uint8)

In [36]:
#Differencing Attack using sum query

sum(db) == sum(pdb)

tensor(1, dtype=torch.uint8)

In [37]:
#Differencing Attack using mean query

(sum(db).float() / len(db)) == (sum(pdb).float() / len(pdb))

tensor(0, dtype=torch.uint8)

In [38]:
#Differencing Attack using threshold

(sum(db).float() > 49) == (sum(pdb).float() > 49)

tensor(1, dtype=torch.uint8)

Project: **Local Differential Privacy**

In [0]:
db, pdbs = create_db_and_parallels(100)

In [40]:
db

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        1, 0, 1, 1], dtype=torch.uint8)

In [0]:
first_coin_flip = (torch.rand(len(db)) > 0.5)
second_coin_flip = (torch.rand(len(db)) > 0.5)

In [42]:
first_coin_flip

tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 0, 0], dtype=torch.uint8)

In [43]:
second_coin_flip

tensor([0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 1], dtype=torch.uint8)

In [44]:
db * first_coin_flip

tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0], dtype=torch.uint8)

In [45]:
true_result = torch.mean(db.float())
true_result

tensor(0.5200)

Randomized Response (Local Differential Privacy)

- Flip a coin 2 times.

- If the first coin flip is heads, answer honestly.

- If the first coin flip is tails, answer according to the second coin flip(heads for yes, tails for no)!

This is called 'Plausible Deniability'

In [0]:
db, pdbs = create_db_and_parallels(100)

In [47]:
db

tensor([0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1], dtype=torch.uint8)

In [48]:
true_result  

tensor(0.5200)